# Kaggle - Brain Tumor MRI Dataset

You can find the dataset and some informations about on the [Kaggle page](https://www.kaggle.com/datasets/masoudnickparvar/brain-tumor-mri-dataset).

For details on steps below, please see documentation in the *docs* directory.

## General part

In [ ]:
import os
from pathlib import Path
import cv2
import numpy as np

from sklearn.model_selection import train_test_split

In [ ]:
# path management
NOTEBOOK_PATH = Path(ipynbname.path())
PROJECT_ROOT = Path(NOTEBOOK_PATH).resolve().parents[1]
RAW_DIR = PROJECT_ROOT / "data" / "raw"
PREP_DIR = PROJECT_ROOT / "data" / "processed"

CLASSES = ["notumor", "glioma", "meningioma", "pituitary"]

# parameters
IMG_SIZE = 260 
SEED = 42
SPLIT = [0.7, 0.30]  # train - validation -> test data are always in a separated directory

## Preprocessing